In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [37]:

DATA_PATH = 'data'
DATA_PREF = ['Fat_Supply_Quantity','Food_Supply_kcal','Food_Supply_Quantity_kg', 'Protein_Supply_Quantity']
DATA_DESCR = 'Supply_Food_Data_Descriptions'
TARGET_COLUMNS = ['Obesity','Undernourished', 'Confirmed',
'Deaths',
'Recovered',
'Active',
'Population',
'Unit (all except Population)']
def load_data(data_path=DATA_PATH, data_prefixes=DATA_PREF, data_description=DATA_DESCR):
    dfs = []
    target = None
    desc = pd.read_csv(os.path.join(data_path, data_description + '.csv')).set_index('Categories')
    

    for prefix in data_prefixes:
        df = pd.read_csv(os.path.join(data_path, prefix + '_Data.csv')).set_index('Country')
        
        target = df[TARGET_COLUMNS]
        df.drop(columns=TARGET_COLUMNS, inplace=True)
        
        df.columns = ['_'.join([i, prefix]) for i in df.columns]
        
        for cat in desc.index:
            new_cats = desc.loc[cat].str.split(pat=';')['Items']
            
            for new_cat in new_cats:
                df['_'.join([new_cat, prefix])] = df['_'.join([cat, prefix])]
        dfs.append(df)
    
    return pd.concat(dfs), target.drop(columns=TARGET_COLUMNS[-1])

In [38]:
data = load_data()

FileNotFoundError: [Errno 2] File data/Supply_Food_Data_Descriptions.csv does not exist: 'data/Supply_Food_Data_Descriptions.csv'

In [32]:
data[1]
    
    

,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population
Country,,,,,,,
Afghanistan,4.5,29.8,0.029370,0.000576,0.002884,0.025911,38042000.0
Albania,22.3,6.2,0.035129,0.001120,0.027817,0.006193,2858000.0
Algeria,26.6,3.9,0.019589,0.001403,0.010936,0.007250,43406000.0
Angola,6.8,25,0.000223,0.000013,0.000057,0.000153,31427000.0
Antigua and Barbuda,19.1,NaN,0.025773,0.003093,0.019588,0.003093,97000.0
...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),25.2,21.2,0.004128,0.000035,0.001059,0.003033,28516000.0
Vietnam,2.1,9.3,0.000341,0.000000,0.000284,0.000056,95656000.0
Yemen,14.1,38.9,0.000799,0.000151,0.000034,0.000614,29162000.0


In [8]:
def clean_data(data):
    
    threshold = 0.85
    #Dropping columns with missing value rate higher than threshold
    data = data[data.columns[data.isnull().mean() < threshold]]

    #Dropping rows with missing value rate higher than threshold
    data = data.loc[data.isnull().mean(axis=1) < threshold]
    for c in data.columns:
        #object -> float 
        if data[c].dtype == 'object':
            data[c] = data[c].str.replace("<","")
            if len(set(data[c])) <= 3:
                data.drop(columns=c, inplace=True)
            else:
                data[c] = data[c].astype('float64')

    #fill missing values with -1
    return data.fillna(-1)
            

In [9]:
new_X = clean_data(data[0])

In [10]:
new_X

,Alcoholic Beverages_Fat_Supply_Quantity,Animal Products_Fat_Supply_Quantity,Animal fats_Fat_Supply_Quantity,"Aquatic Products, Other_Fat_Supply_Quantity",Cereals - Excluding Beer_Fat_Supply_Quantity,Eggs_Fat_Supply_Quantity,"Fish, Seafood_Fat_Supply_Quantity",Fruits - Excluding Wine_Fat_Supply_Quantity,Meat_Fat_Supply_Quantity,Miscellaneous_Fat_Supply_Quantity,...,Cassava and products_Protein_Supply_Quantity,Cloves_Protein_Supply_Quantity,Cocoa Beans and products_Protein_Supply_Quantity,Coconut Oil_Protein_Supply_Quantity,Coconuts - Incl Copra_Protein_Supply_Quantity,Honey_Protein_Supply_Quantity,Infant food_Protein_Supply_Quantity,Nuts and products_Protein_Supply_Quantity,Onions_Protein_Supply_Quantity,Sugar beet_Protein_Supply_Quantity
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,21.6397,6.2224,0.0,8.0353,0.6859,0.0327,0.4246,6.1244,0.0163,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
Albania,0.0,32.0002,3.4172,0.0,2.6734,1.6448,0.1445,0.6418,8.7428,0.0170,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
Algeria,0.0,14.4175,0.8972,0.0,4.2035,1.2171,0.2008,0.5772,3.8961,0.0439,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
Angola,0.0,15.3041,1.3130,0.0,6.5545,0.1539,1.4155,0.3488,11.0268,0.0308,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
Antigua and Barbuda,0.0,27.7033,4.6686,0.0,3.2153,0.3872,1.5263,1.2177,14.3202,0.0898,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela (Bolivarian Republic of),-1.0,-1.0000,-1.0000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,27.4545,27.4545,27.4545,27.4545,27.4545,27.4545,27.4545,27.4545,27.4545,27.4545
Vietnam,-1.0,-1.0000,-1.0000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,29.5617,29.5617,29.5617,29.5617,29.5617,29.5617,29.5617,29.5617,29.5617,29.5617
Yemen,-1.0,-1.0000,-1.0000,-1.0,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,39.9831,39.9831,39.9831,39.9831,39.9831,39.9831,39.9831,39.9831,39.9831,39.9831


In [11]:
for c in new_X.columns:
    assert new_X[c].dtype == 'float64'
        


In [12]:
from sklearn.decomposition import PCA
def apply_PCA(data, n_components=100):
    pca = PCA(n_components=n_components)
    pca.fit(data.to_numpy())
    
    print (pca.explained_variance_ratio_)
    return pca.transform(data)
    
    

In [13]:
pca_X = apply_PCA(new_X)

[4.04687942e-01 3.19543511e-01 2.47788437e-01 1.49650834e-02
 6.56003631e-03 2.88641114e-03 1.56978900e-03 2.63688764e-04
 2.32130564e-04 2.28338531e-04 1.88099647e-04 1.76999000e-04
 1.21688710e-04 1.19188481e-04 8.95680123e-05 8.48106810e-05
 7.29160420e-05 5.36545038e-05 4.99881844e-05 3.28898689e-05
 3.14498963e-05 3.00583472e-05 2.35785568e-05 2.20236499e-05
 2.19603620e-05 2.02994819e-05 1.66440946e-05 1.46182756e-05
 1.32074025e-05 1.21235442e-05 1.09333057e-05 8.62974528e-06
 7.64057342e-06 7.30825747e-06 5.43758395e-06 5.07242143e-06
 4.20966278e-06 2.88445081e-06 2.63515025e-06 2.60529265e-06
 2.39181561e-06 2.18874965e-06 2.16779203e-06 1.64322511e-06
 1.58481042e-06 1.41394306e-06 1.21186011e-06 1.13761012e-06
 9.48127427e-07 9.30555567e-07 6.59651892e-07 4.97316696e-07
 4.68644971e-07 4.63799420e-07 4.43053409e-07 3.68187547e-07
 3.10559179e-07 3.09300382e-07 2.55377644e-07 2.42265583e-07
 2.36780087e-07 2.11101934e-07 2.10985464e-07 2.04886211e-07
 1.79082302e-07 1.455198

In [15]:
def replaceNaN(dataframe, replaceby = "mean"):
    if (replaceby == "mean"):
        return dataframe.fillna(dataframe.mean())
    
    elif (replaceby == "median"):
        return dataframe.fillna(dataframe.median())
    
    elif (replaceby == "mostfrq"):
        return dataframe.fillna(dataframe.mode().iloc[0])
    
    else:return dataframe.dropna()
    
# Replce any nonnumaric char from a column.    
def replce_char(dataframe,collumn, char=r'^<'):
    dataframe[collumn] =  dataframe[collumn].replace(regex=char, value='')
    dataframe[collumn] = pd.to_numeric(dataframe[collumn],errors='coerce')
    return dataframe
    
    
    
# Scale the data using standard scaler or max min scaler. Menstion the names to scale. 
def scale(dataframe, coulumns, scaler="ss"):
    if (scaler == "mm"):
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        dataframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
        return dataframe
    elif (scaler == "ss"):
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        datqframe[coulumns] = scaler.fit_transform(dataframe[coulumns])
        return dataframe
    
    else:return dataframe
        
    


In [18]:
Protein_Supply_DF = pd.read_csv(os.path.join(DATA_PATH,"Food_Supply_Quantity_kg_Data.csv"))
Protein_Supply_DF = replaceNaN(Protein_Supply_DF, replaceby="mostfrq")



In [19]:
NOT_IPMORTANT_COLUMNS = ["Country", 'Unit (all except Population)', 'Population','Obesity', 'Undernourished', 'Deaths','Recovered','Active','Confirmed']

clm = Protein_Supply_DF.columns.drop(NOT_IPMORTANT_COLUMNS)

Protein_Supply_DF = replce_char(Protein_Supply_DF,"Undernourished", char=r'^<')
scaled_df = scale(Protein_Supply_DF, clm, scaler="mm")


In [20]:
scaled_df = scaled_df.sample(frac=1,random_state=1).reset_index(drop=True)
FEATURE_CLS = ['Obesity', 'Undernourished', 'Deaths','Recovered','Active','Confirmed']
FEATURE = ['Obesity', 'Undernourished', 'Deaths','Recovered' ]

ALL_OUTPUT = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
       'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
       'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
       'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
       'Miscellaneous']
GOOD_OUTPUT=['Animal Products', 'Animal fats','Vegetal Products','Vegetable Oils' ]

In [21]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(scaled_df, test_size=0.2, random_state=42)

In [23]:
from keras.models import Sequential
from keras.layers import Dense
class NN_model():
    def __init__(self, n_features=23, n_output=4):

        model = Sequential()
        model.add(Dense(20, input_dim=n_features, kernel_initializer='random_uniform', activation='relu'))
        model.add(Dense(n_output, input_dim=20, kernel_initializer='random_uniform', activation='softmax'))
    def fit(self, X, y):
        return model.fit(X, y)
        

In [24]:
from sklearn.linear_model import LinearRegression
from keras import backend as K

def Regression(features, response, data, val):
    def r2_keras(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
        return ( 1 - SS_res/(SS_tot + K.epsilon()) )
    X = data[features]
    Y = data[response]
    X_val = val[features]
    Y_val = val[response]
    print (X.shape, Y.shape)
    model = Sequential()
    n_features = len(features)
    n_output = len(response)
    print (n_output)
    model.add(Dense(20, input_dim=n_features, kernel_initializer='random_uniform', activation='relu'))
    model.add(Dense(n_output, input_dim=20, kernel_initializer='random_uniform', activation='softmax'))  
    model.compile(loss=r2_keras, optimizer="sgd",metrics=[r2_keras])
    model.fit(X, Y, validation_data=(X_val, Y_val), epochs=5)
    return model
    
    
def TestScore(features, response, data, model):
    print('Output: ', response)
    X = data[features]
    Y = data[response]
    return model.predict(X,Y)
    
def PlotRegression(features, response, data, model):
    import matplotlib.pyplot as plt
    X = data[features]
    Y = data[response]
    y_pred = model.predict(X)
    plt.scatter(X, Y, s=10)
    plt.plot(X, y_pred, color='r')
    plt.show()

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
X = pd.DataFrame(scaled_df[FEATURE_CLS])
y = pd.DataFrame(scaled_df[GOOD_OUTPUT])
model = Regression(FEATURE_CLS, GOOD_OUTPUT, train_set, test_set)
#score, acc = TestScore(FEATURE_CLS, GOOD_OUTPUT,test_set, model)
# scores = []
# kfold = KFold(n_splits=3, shuffle=True, random_state=1)
# for i, (train, test) in enumerate(kfold.split(X, y)):
#     model.fit(X.iloc[train,:], y.iloc[train,:])
#     score = model.score(X.iloc[test,:], y.iloc[test,:])
#     scores.append(score)
# print(scores)

(136, 6) (136, 4)
4
Epoch 1/5
5/5 [==============================] - 0s 42ms/step - loss: -0.2269 - r2_keras: -0.2571 - val_loss: -0.3773 - val_r2_keras: -0.3219
Epoch 2/5
5/5 [==============================] - 0s 13ms/step - loss: -0.3646 - r2_keras: -0.4104 - val_loss: -0.6343 - val_r2_keras: -0.4329
Epoch 3/5
5/5 [==============================] - 0s 9ms/step - loss: -1.2351 - r2_keras: -1.7088 - val_loss: -3.6228 - val_r2_keras: -2.7276
Epoch 4/5
5/5 [==============================] - 0s 9ms/step - loss: -4.0750 - r2_keras: -4.2359 - val_loss: -3.9863 - val_r2_keras: -3.0852
Epoch 5/5
5/5 [==============================] - 0s 9ms/step - loss: -4.3689 - r2_keras: -4.2868 - val_loss: -4.0166 - val_r2_keras: -3.1116


In [26]:
top8output = ['Animal Products', 'Animal fats', 'Meat', 'Cereals - Excluding Beer','Vegetables','Vegetal Products','Vegetable Oils', 'Oilcrops' ]
top23output = ['Alcoholic Beverages', 'Animal Products', 'Animal fats',
          'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
          'Fish, Seafood', 'Fruits - Excluding Wine', 'Meat',
          'Milk - Excluding Butter', 'Offals', 'Oilcrops', 'Pulses', 'Spices',
          'Starchy Roots', 'Stimulants', 'Sugar Crops', 'Sugar & Sweeteners',
          'Treenuts', 'Vegetal Products', 'Vegetable Oils', 'Vegetables',
          'Miscellaneous']
model = Regression(FEATURE_CLS, top8output, train_set, test_set)


(136, 6) (136, 8)
8
Epoch 1/5
5/5 [==============================] - 0s 20ms/step - loss: -0.5842 - r2_keras: -0.6020 - val_loss: -0.6788 - val_r2_keras: -0.5830
Epoch 2/5
5/5 [==============================] - 0s 8ms/step - loss: -0.5990 - r2_keras: -0.6105 - val_loss: -0.7022 - val_r2_keras: -0.6019
Epoch 3/5
5/5 [==============================] - 0s 7ms/step - loss: -0.6271 - r2_keras: -0.6736 - val_loss: -0.7389 - val_r2_keras: -0.6314
Epoch 4/5
5/5 [==============================] - 0s 7ms/step - loss: -0.6653 - r2_keras: -0.6578 - val_loss: -0.8098 - val_r2_keras: -0.6892
Epoch 5/5
5/5 [==============================] - 0s 7ms/step - loss: -0.7571 - r2_keras: -0.8120 - val_loss: -1.0901 - val_r2_keras: -0.9298


In [27]:
model = Regression(FEATURE_CLS, top23output, train_set, test_set)


(136, 6) (136, 23)
23
Epoch 1/5
5/5 [==============================] - 0s 21ms/step - loss: -0.6307 - r2_keras: -0.6512 - val_loss: -0.6161 - val_r2_keras: -0.6194
Epoch 2/5
5/5 [==============================] - 0s 9ms/step - loss: -0.6334 - r2_keras: -0.6076 - val_loss: -0.6171 - val_r2_keras: -0.6202
Epoch 3/5
5/5 [==============================] - 0s 8ms/step - loss: -0.6320 - r2_keras: -0.6489 - val_loss: -0.6182 - val_r2_keras: -0.6210
Epoch 4/5
5/5 [==============================] - 0s 9ms/step - loss: -0.6340 - r2_keras: -0.6329 - val_loss: -0.6193 - val_r2_keras: -0.6218
Epoch 5/5
5/5 [==============================] - 0s 8ms/step - loss: -0.6357 - r2_keras: -0.6356 - val_loss: -0.6205 - val_r2_keras: -0.6227


In [39]:
from dash_app.utils import NutritionData

all_data_file = [os.path.join(DATA_PATH, "Food_Supply_Quantity_kg_Data.csv"),
                      os.path.join(DATA_PATH, "Fat_Supply_Quantity_Data.csv"),
                      os.path.join(DATA_PATH, "Food_Supply_kcal_Data.csv"),
                      os.path.join(DATA_PATH, "Protein_Supply_Quantity_Data.csv")]


for data in all_data_file:
    data_obj = NutritionData(data_path=data)
    model = NutritionModel(dataset=data_obj.dataset)
    model.split_data()
    train, test = model.train, model.test
    

FileNotFoundError: [Errno 2] File data/Food_Supply_Quantity_kg_Data.csv does not exist: 'data/Food_Supply_Quantity_kg_Data.csv'